In [ ]:
#import tensorflow.compat.v1 as tf
#tf.enable_eager_execution(tf.ConfigProto(log_device_placement=False))
#tf.test.gpu_device_name()

import numpy as np
from sklearn.model_selection import KFold
from itertools import product
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Masking, LSTM, GRU
from sklearn.model_selection import KFold
# import early stopping
from tensorflow.keras.callbacks import EarlyStopping
import pickle
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import class_weight

from tensorflow.keras.layers import Input, Concatenate, Permute, Reshape, Multiply, Lambda, Add
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

from tensorflow.keras.regularizers import l2


In [ ]:
from sklearn.model_selection import train_test_split

def train_and_evaluate_warped(dataset_names, create_model_fn, class_weights):

    dataset_results = {}

    outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

    # Find best dataset to train and test model on
    #dataset_names = ['X_smoothed_mean_norm']

    for dataset_name in dataset_names:

        # Initialize variables to track the best model
        best_model = None
        best_performance = 0
        best_model_details = ''


         # Load in warped dataset from pickle
        with open(f'drive/MyDrive/Pickles/{dataset_name}.pickle', 'rb') as handle:
            dataset = pickle.load(handle)

        # Load in smoothed mean dataset from pickle
        with open(f'drive/MyDrive/Pickles/X_smoothed_mean_norm.pickle', 'rb') as handle:
            unwarped = pickle.load(handle)

        dataset = np.array(dataset)
        unwarped = np.array(unwarped)

        dataset_results[dataset_name] = {}

        print(f'Working on dataset: {dataset_name}')

        accuracies = []
        sensitivities = []
        false_positive_rates = []
        specificities = []
        precisions = []
        performance = []

        fold = 0

        unwarped_split = list(outer_cv.split(unwarped))

        for train_index, test_index in outer_cv.split(dataset):

            unwarped_train_index, unwarped_test_index = unwarped_split[fold]

            # Print current progress
            print(f'Working on fold: {fold}')
            fold += 1

            X_train, X_test_warped = dataset[train_index], dataset[test_index]
            Y_train, Y_test_warped = abnormal_encoded_warped[train_index], abnormal_encoded_warped[test_index]

            X_test = unwarped[unwarped_test_index]
            Y_test = abnormal_encoded[unwarped_test_index]

            X_train_unwarped, X_val, Y_train_unwarped, Y_val = train_test_split(
                unwarped[unwarped_train_index], abnormal_encoded[unwarped_train_index], test_size=0.2, random_state=42)

            X_train = X_train.astype('float32')
            Y_train = Y_train.astype('float32')
            X_test = X_test.astype('float32')
            Y_test = Y_test.astype('float32')

            model = create_model_fn(X_train.shape[1:])
            early_stopping = EarlyStopping(monitor='val_loss', patience=3)

            if class_weights:

                Y_train_classes = np.argmax(Y_train, axis=1)

                # Compute class weights
                cw = class_weight.compute_class_weight('balanced',
                                                    classes=np.unique(Y_train_classes),
                                                    y=Y_train_classes)

                class_weights_dict = dict(enumerate(cw))

                #Fit the model
                print('Fitting model')
                model.fit(X_train, Y_train, epochs=15, batch_size=16, callbacks=[early_stopping], verbose=1, class_weight=class_weights_dict, validation_data = (X_val, Y_val))

            else:
                #Fit the model
                print('Fitting model')
                model.fit(X_train, Y_train, epochs=15, batch_size=16,  callbacks=[early_stopping], verbose=1, validation_data = (X_val, Y_val))

            # Predict the test set
            print('Predicting test set')
            Y_pred = model.predict(X_test)

            Y_pred_classes = np.argmax(Y_pred, axis=1)
            Y_test_classes = np.argmax(Y_test, axis=1)

            # Calulate accuracy, sensitivity, false positive rate, specificity and precision
            accuracy = (accuracy_score(Y_test_classes, Y_pred_classes))
            sensitivity = (recall_score(Y_test_classes, Y_pred_classes, average='macro'))
            false_positive_rate = (1 - specificity_score(Y_test, Y_pred))
            specificity = (specificity_score(Y_test, Y_pred))
            precision = (precision_score(Y_test_classes, Y_pred_classes, average='macro'))

            # Calculate the custom performance metric
            model_performance = custom_performance_metric(accuracy, specificity, sensitivity, precision, false_positive_rate)

            # Calulate accuracy, sensitivity, false positive rate, specificity and precision
            accuracies.append(accuracy)
            sensitivities.append(sensitivity)
            false_positive_rates.append(false_positive_rate)
            specificities.append(specificity)
            precisions.append(precision)
            performance.append(model_performance)
            
            print(f"Model's custom performance metric: {model_performance}")

            # Update the best model if current model is better
            '''if model_performance > best_performance:
                best_performance = model_performance
                best_model = model
                best_model_details = f'{dataset_name}_fold_{fold}'''

        avg_accuracy = np.mean(accuracies)
        avg_sensitivity = np.mean(sensitivities)
        avg_false_positive_rate = np.mean(false_positive_rates)
        avg_specificity = np.mean(specificities)
        avg_precision = np.mean(precisions)
        avg_performance = np.mean(performance)

        std_accuracy = np.std(accuracies)
        std_sensitivity = np.std(sensitivities)
        std_false_positive_rate = np.std(false_positive_rates)
        std_specificity = np.std(specificities)
        std_precision = np.std(precisions)
        std_performance = np.std(performance)

        dataset_results[dataset_name]['Accuracy'] = (avg_accuracy, std_accuracy)
        dataset_results[dataset_name]['Sensitivity'] = (avg_sensitivity, std_sensitivity)
        dataset_results[dataset_name]['False Positive Rate'] = (avg_false_positive_rate, std_false_positive_rate)
        dataset_results[dataset_name]['Specificity'] = (avg_specificity, std_specificity)
        dataset_results[dataset_name]['Precision'] = (avg_precision, std_precision)
        dataset_results[dataset_name]['Performance'] = (avg_performance, std_performance)

        '''if best_model:
            model_save_path = f'models/best_model_{best_model_details}.keras'
            print(f'Saving best model to {model_save_path}')
            best_model.save(model_save_path)
            print(f'Best Model Details: {best_model_details}, Performance: {best_performance}')'''
            
        # Delete dataset to free up memory
        del dataset
        del Y_pred

    for dataset_name, results in dataset_results.items():
        print(f'Dataset: {dataset_name}')
        for metric, (avg, std) in results.items():
            print(f'{metric}: {avg} +/- {std}')
        print('\n')

    return dataset_results

In [ ]:
 # Load in dataset from pickle
with open('drive/MyDrive/Pickles/labels_magnitude_warped.pickle', 'rb') as handle:
    abnormal_encoded_warped = pickle.load(handle)

# Use the sklearn OneHotEncoder to one-hot encode the data
from sklearn.preprocessing import OneHotEncoder

# Create the OneHotEncoder object
onehot_encoder = OneHotEncoder(sparse=False)
# One-hot encode the data
abnormal_encoded_warped = onehot_encoder.fit_transform(np.array(abnormal_encoded_warped).reshape(-1, 1))

print(abnormal_encoded_warped)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

def create_improved_cnn_model(shape, filters=32, kernel_size=3, dropout_rate=0.5):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))  # Adjust the input_shape based on your data
    model.add(Conv1D(filters, kernel_size, activation='relu', kernel_regularizer=l2(0.01), input_shape=shape))
    model.add(BatchNormalization())  # Normalize the activations of the previous layer at each batch
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout_rate))  # Apply dropout
    model.add(Flatten())
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))  # Apply regularization
    model.add(Dropout(dropout_rate))  # Apply dropout again
    model.add(Dense(3, activation='softmax'))  # Assuming a 3-class classification problem
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
warped_cnn_dataset_result_ncw = train_and_evaluate_warped(['X_magnitude_warped'], create_improved_cnn_model, False)